In [10]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os, netCDF4, xarray

os.chdir('/Users/hausfath/Desktop/Climate Science/Stripe/Learning Rates/learning_rates')
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [30]:
'''
Calculate the number of doublings and cumulative cost needed reach a particular price target 
as a function of initial volume, initial price, target price, and learning rate
'''

df = pd.DataFrame()
df['doublings'] = np.arange(0,100,1)
init_vol = 0.01
target = 100

results = pd.DataFrame()

for init_price in np.arange(300, 800, 100):
    for learn in np.arange(0.05, 0.21, 0.01):
        df['price']=init_price*(1-learn)**df['doublings']
        df['cum_install'] = init_vol * 2**df.index
        df['install'] = df['cum_install'].diff()
        df.loc[df['doublings'] == 0, 'install'] = init_vol
        df['cost'] = df['install']*df['price']/1000
        df['cum_cost'] = df['cost'].cumsum()
        df['learn'] = learn
        df['init_price'] = init_price
        temp = df.iloc[(df['price']-target).abs().argsort()[:1]]
        results = pd.concat([results, temp])
results

,doublings,price,cum_install,install,cost,cum_cost,learn,init_price
21,21,102.17,20971.52,10485.76,1071.31,2261.66,0.05,300
18,18,98.50,2621.44,1310.72,129.10,275.81,0.06,300
15,15,101.01,327.68,163.84,16.55,35.79,0.07,300
13,13,101.48,81.92,40.96,4.16,9.10,0.08,300
12,12,96.74,40.96,20.48,1.98,4.40,0.09,300
...,...,...,...,...,...,...,...,...
11,11,102.84,20.48,10.24,1.05,2.60,0.16,700
10,10,108.61,10.24,5.12,0.56,1.40,0.17,700
10,10,96.21,10.24,5.12,0.49,1.26,0.18,700
9,9,105.07,5.12,2.56,0.27,0.70,0.19,700


In [36]:
'''
Calculate the cumulative cost needed to get to a particular price threshold
for a given learning rate, initial volume, and initial price.
'''

df = pd.DataFrame()
doublings_per_year = 1
max_learning = 0.2
price_threshold = 100.

results = pd.DataFrame()
for init_price in np.arange(1000, 8000, 1000):
    for init_volume in np.arange(100,10000,50):
        init_volume = init_volume
        df['doublings'] = np.arange(0,100,doublings_per_year)
        df['price']=init_price*(1-max_learning)**df['doublings']
        df['cum_install'] = init_volume/10**6 * 2**df['doublings']
        df['install'] = df['cum_install'].diff()
        df.loc[df['doublings'] == 0, 'install'] = init_volume/10**6
        df['cost'] = df['install']*df['price']/1000
        df['cum_cost'] = df['cost'].cumsum()
        df['init_volume'] = init_volume
        df['init_price'] = init_price
        temp = df.iloc[(df['price']-price_threshold).abs().argsort()[:1]]
        results = pd.concat([results, temp])
#results.to_csv('learning_costs.csv')
results

,doublings,price,cum_install,install,cost,cum_cost,init_volume,init_price
10,10,107.37,0.10,0.05,0.01,0.01,100,1000
10,10,107.37,0.15,0.08,0.01,0.02,150,1000
10,10,107.37,0.20,0.10,0.01,0.03,200,1000
10,10,107.37,0.26,0.13,0.01,0.04,250,1000
10,10,107.37,0.31,0.15,0.02,0.04,300,1000
...,...,...,...,...,...,...,...,...
19,19,100.88,5111.81,2555.90,257.84,687.55,9750,7000
19,19,100.88,5138.02,2569.01,259.16,691.08,9800,7000
19,19,100.88,5164.24,2582.12,260.49,694.61,9850,7000
19,19,100.88,5190.45,2595.23,261.81,698.13,9900,7000


In [38]:
sorted = results.iloc[(results['cum_cost']-20).abs().argsort()]
sorted[sorted['init_price']==2000][:3]

,doublings,price,cum_install,install,cost,cum_cost,init_volume,init_price
13,13,109.95,81.51,40.76,4.48,11.94,9950,2000
13,13,109.95,81.10,40.55,4.46,11.88,9900,2000
13,13,109.95,80.69,40.35,4.44,11.82,9850,2000
